## Cap to 1000 words

In [2]:
data = ["train", "test", "valid"]
for name in data:
    with open("data/hd/raw/" + name + ".wp_target") as f:
        stories = f.readlines()
    stories = [" ".join(i.split()[0:1000]) for i in stories]
    with open("data/hd/prepro/" + name + ".wp_target", "w") as o:
        for line in stories:
            o.write(line.strip() + "\n")

KeyboardInterrupt: 

## Initiate Tokenizer

In [5]:
from transformers import OPTForCausalLM, AutoTokenizer

model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(tokenizer)

/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPT2TokenizerFast(name_or_path='facebook/opt-350m', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}


In [7]:
import pickle

def tokenize_and_save(file_prefix, batch_size=100):
    prompts_file = f"data/hd/prepro/{file_prefix}.wp_source"
    stories_file = f"data/hd/prepro/{file_prefix}.wp_target"
    output_file = f'data/hd/prepro/tokenized/tokenized_{file_prefix}_data.pkl'
    
    with open(prompts_file, encoding="utf-8") as p, open(stories_file, encoding="utf-8") as f, open(output_file, 'wb') as d:
        while True:
            prompts = [next(p, None) for _ in range(batch_size)]
            stories = [next(f, None) for _ in range(batch_size)]
            # Break if the first item is None, indicating end of file
            if prompts[0] is None or stories[0] is None:
                break
            
            # Filter out None values in case of mismatched lengths (shouldn't happen with well-formed data)
            prompts = [prompt for prompt in prompts if prompt]
            stories = [story for story in stories if story]
            
            # Tokenize batch
            tokenized_batch = tokenizer(prompts, stories, padding=True, max_length=1024, truncation=True, return_tensors="pt")
            
            # Save tokenized batch immediately to reduce memory usage
            pickle.dump(tokenized_batch, d)

# Example usage
for name in ["train", "test", "valid"]:
    tokenize_and_save(name, batch_size=100)